
## Set detault catalog and schema

In [0]:
USE amitabh_arora_catalog.bright_data_amazon_reviews;

## Display dataset

In [0]:
SELECT *
FROM amazon_reviews
LIMIT 10

## Using ai_analyze_sentiment()

In [0]:
SELECT
  TIMESTAMP,
  ai_analyze_sentiment(REVIEW_TEXT) as sentiment,
  REVIEW_TEXT,
  PRODUCT_NAME,
  PRODUCT_RATING,
  PRODUCT_RATING_OBJECT,
  PRODUCT_RATING_COUNT,
  PRODUCT_REVIEWS_NUMBER
FROM amazon_reviews
LIMIT 10

For example this review was classified as positive:

```
First thing I did was watch several videos on how to use this item.  This soup
maker works as well as much more expensive ones shown on those videos.  I was
surprised at how quickly it makes a great tasting soup.  The portion is great
for the two of us with a little bit left over.
```

## Using ai_query()

ai_query() will allow you to specify your own model, prompt, and structured output response. It enables high throughput performance with no need for complex configuration.

Lets take a look at sample data below:
```I got soup. It took only 20 minutes to make some good soup. The noises
it makes when blending are somewhat terrifying, but it gives a little
beep to warn you before it does that. It made three or four large
servings of soup. It's a single layer of steel, so the outside gets
pretty hot. It can be hard to unplug the lid without knocking the
blender against the side, which is not a pleasant sound. The soup was
good, and its recipes look delicious, but I'm still determining if I'll
use it often. Twenty minutes of scary noises from the kitchen when I
need comfort food could be better for me. But if you aren't sensitive to
loud sounds, it does exactly what it says it does.
```

Next, we should decide what we want in the output. Ideally, we could extract the critical opinions, record the sentiment, and classify the mention into a relevant category. The response might look like this:
```
[
 {
	'Classification': 'EFFICIENCY',
  'Comment': 'only 20 minutes',
  'Sentiment': 'Positive'
 },
 ...
]
```

To produce this output, we’ll construct a SQL query that uses ai_query() to call an LLM. We’ll create a custom prompt that instructs the LLM to classify reviews into our desired categories and provide a sample review and output to improve the quality of the response. We'll also specify a responseFormat, which defines a structured output that we want the result returned in.


In [0]:
SELECT
timestamp,
 ai_query(
   "databricks-meta-llama-3-3-70b-instruct",
   request => concat("You are an opinion mining service. Given a piece of text, output an array of json results that extracts key user opinions, a classification, and a Positive, Negative, Neutral, or Mixed sentiment about that subject. Classifications must be one of the following answers: QUALITY, SERVICE, EFFICIENCY, DURABILITY, BATTERY LIFE, DESIGN, SAFETY, USABILITY, PRICE, COMFORT, OTHER. You cannot hallucinate your own classification category.

 Examples below:

 DOCUMENT
 I got soup. It really did take only 20 minutes to make some pretty good soup.The noises it makes when it's blending are somewhat terrifying, but it gives a little beep to warn you before it does that.It made three or four large servings of soup.It's a single layer of steel, so the outside gets pretty hot. It can be hard to unplug the lid without knocking the blender against the side, which is not a pleasant sound.The soup was good and the recipes it comes with look delicious, but I'm not sure I'll use it often. 20 minutes of scary noises from the kitchen when I already need comfort food is not ideal for me. But if you aren't sensitive to loud sounds it does precisely what it says it does.

 RESULT
 [
 {'Classification': 'EFFICIENCY', 'Comment': 'only 20 minutes','Sentiment': 'Positive'},
 {'Classification': 'QUALITY','Comment': 'pretty good soup','Sentiment': 'Positive'},
 {'Classification': 'QUALITY', 'Comment': 'noises it makes when it's blending are somewhat terrifying', 'Sentiment': 'Negative'},
 {'Classification': 'SAFETY','Comment': 'outside gets pretty hot','Sentiment': 'Negative'},
 {'Classification': 'DESIGN','Comment': 'Hard to unplug the lid without knocking the blender against the side, which is not a nice sound', 'Sentiment': 'Negative'}
 ]

DOCUMENT\n",
       REVIEW_TEXT,
       '\n\nRESULT\n'
     ),
     responseFormat => '{
    "type": "json_schema",
    "json_schema": {
      "name": "opinion_mining_schema",
      "schema": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "classification": { "type": "string" },
              "comment": { "type": "string" },
              "sentiment": { "type": "string" }
            }
          }
      },
      "strict": true
    }
  }'
   ) as extracted_opinions,
   REVIEW_TEXT
 FROM
   amazon_reviews
 limit 10

## Optimize the process

Now that we know this works, we can make a few improvements.

1. Extract the JSON columns.
2. Filter out any hallucinated values that don’t match our desired classifications.
3. Create a materialized view to store these results so we don’t have to recompute the LLM outputs each time we want to look at our dashboard. You can refresh the materialized view whenever you want to incrementally compute more review opinions without needing to rescan the entire dataset.

**NOTE: Execute this query. It may take some time to complete, so make a cup of coffee or tea until the query completes.**

In [0]:
CREATE MATERIALIZED VIEW opinion_mining_mv
AS
WITH reviews AS (
  -- calculate opinions
  SELECT
    *,
    ai_query(
      "databricks-meta-llama-3-3-70b-instruct",
      request => concat(
        "You are an opinion mining service. Given a piece of text, output an array of json results that extracts key user opinions, a classification, and a Positive, Negative, Neutral, or Mixed sentiment about that subject. Classifications must be one of the following answers: QUALITY, SERVICE, EFFICIENCY, DURABILITY, BATTERY LIFE, DESIGN, SAFETY, USABILITY, PRICE, COMFORT, OTHER. You cannot hallucinate your own classification category.

        Examples below:

        DOCUMENT
        I got soup. It really did take only 20 minutes to make some pretty good soup.The noises it makes when it's blending are somewhat terrifying, but it gives a little beep to warn you before it does that.It made three or four large servings of soup.It's a single layer of steel, so the outside gets pretty hot. It can be hard to unplug the lid without knocking the blender against the side, which is not a pleasant sound.The soup was good and the recipes it comes with look delicious, but I'm not sure I'll use it often. 20 minutes of scary noises from the kitchen when I already need comfort food is not ideal for me. But if you aren't sensitive to loud sounds it does precisely what it says it does.

        RESULT
        [
          {'Classification': 'EFFICIENCY', 'Comment': 'only 20 minutes','Sentiment': 'Positive'},
          {'Classification': 'QUALITY','Comment': 'pretty good soup','Sentiment': 'Positive'},
          {'Classification': 'QUALITY', 'Comment': 'noises it makes when it's blending are somewhat terrifying', 'Sentiment': 'Negative'},
          {'Classification': 'SAFETY','Comment': 'outside gets pretty hot','Sentiment': 'Negative'},
          {'Classification': 'DESIGN','Comment': 'Hard to unplug the lid without knocking the blender against the side, which is not a nice sound', 'Sentiment': 'Negative'}
        ]

        DOCUMENT\n",
        REVIEW_TEXT,
        '\n\nRESULT\n'
      ),
      responseFormat => '{
        "type": "json_schema",
        "json_schema": {
          "name": "opinion_mining_schema",
          "schema": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "classification": { "type": "string" },
                "comment": { "type": "string" },
                "sentiment": { "type": "string" }
              }
            }
          },
          "strict": true
        }
      }'
    ) AS extracted_opinions
  FROM
    amazon_reviews
), opinions AS (
  -- explode json into columns
  SELECT
    opinion_col.classification AS Classification,
    opinion_col.comment AS Comment,
    opinion_col.sentiment AS Sentiment,
    *
  FROM
    reviews LATERAL VIEW explode(
      from_json(
        extracted_opinions,
        'array<struct<classification:string, comment:string, sentiment:string>>'
      )
    ) AS opinion_col
)
SELECT
  *
FROM
  opinions
WHERE
  classification IN ("QUALITY", "SERVICE", "EFFICIENCY", "DURABILITY", "BATTERY LIFE", "DESIGN", "SAFETY", "USABILITY", "PRICE", "COMFORT", "OTHER")